# Handling Job Dependencies that Can Fail

In this tutorial, we will demonstrate how to handle missing references in JobFlow. This is useful when you have jobs that may fail, but you still want to proceed with the workflow.

First, we import the necessary modules and define a job that can fail based on an input parameter.


In [2]:
from jobflow import Flow, OnMissing, job, run_locally


@job
def func(fail: bool = False):
    """Failable function."""
    if fail:
        raise ValueError("An error occurred.")
    return 1

Next, we define a job that collects the outputs of other jobs. This job can handle missing references.

**Note:** You must explicitly define how missing references are handled in each job.
setting `on_missing_refs` to `OnMissing.None` will only provide a `None` whenever an output is missing.
you must handle those `None` values in your job.

In [3]:
@job
def collect(job_outputs):
    """Job that allows some parents to fail."""
    total = 0
    for output in job_outputs:
        if output is None:
            continue
        total += output
    if total < 1:
        raise ValueError("No enough finished parents.")
    return total

Now, we create instances of the `func` job, one of which will fail.
Then, we create an instance of the `collect` job and pass the outputs of the `func` jobs to it.

By setting the `on_missing_refs` parameter to `OnMissing.None`, and handling the `None` values in the `collect` job, we can proceed with the workflow even if some references are missing.

In [4]:
job1, job2, job3 = func(), func(), func(fail=True)
job_outputs = [job1.output, job2.output, job3.output]
collect_job = collect(job_outputs)
collect_job.config.on_missing_references = OnMissing.NONE

As the workflow is running, `job1` and `job2` will each return 1, while job3 will fail. 
Since `collect_job` has `on_missing_references` set to `OnMissing.NONE`, it proceeds despite the missing output from `job3`. 


In [5]:
flow = Flow([job1, job2, job3, collect_job])
res = run_locally(flow)
n_finished = 2
assert res[collect_job.uuid][1].output == n_finished

2024-12-13 17:28:10,890 INFO Started executing jobs locally
2024-12-13 17:28:10,991 INFO Starting job - func (12fc77ef-233e-4bee-a36c-7f61dc5badf9)
2024-12-13 17:28:10,996 INFO Finished job - func (12fc77ef-233e-4bee-a36c-7f61dc5badf9)
2024-12-13 17:28:10,997 INFO Starting job - func (626268f0-ecb0-4fa2-bf6d-0bb9dd72586c)
2024-12-13 17:28:10,999 INFO Finished job - func (626268f0-ecb0-4fa2-bf6d-0bb9dd72586c)
2024-12-13 17:28:11,000 INFO Starting job - func (ba559709-99f7-4ec3-9fe7-6804a002ff0a)
2024-12-13 17:28:11,002 INFO func failed with exception:
Traceback (most recent call last):
  File "/home/jmmshn/miniconda3/envs/af/lib/python3.10/site-packages/jobflow/managers/local.py", line 114, in _run_job
    response = job.run(store=store)
  File "/home/jmmshn/miniconda3/envs/af/lib/python3.10/site-packages/jobflow/core/job.py", line 600, in run
    response = function(*self.function_args, **self.function_kwargs)
  File "/tmp/ipykernel_298791/2449992254.py", line 7, in func
    raise Valu